In [1]:
# %pip install torch
# %pip install pymoo
# %pip install pywin32

In [ ]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import *
from FlashOperation.Refrig2DrumHeatExConstr1 import Refrig2DrumConstraintHeatExConstrforNN, Refrig2Drumproblem

Added to sys.path: C:\Users\conno\OneDrive\Desktop\Aspen\Aspen
C:\Users\conno\OneDrive\Desktop\Aspen\Aspen
['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.1008.0_x64__qbz5n2kfra8p0\\python313.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.1008.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.1008.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.13_3.13.1008.0_x64__qbz5n2kfra8p0', '', 'C:\\Users\\conno\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages', 'C:\\Users\\conno\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\win32', 'C:\\Users\\conno\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\sit

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import time

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"



print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstrforNN(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=False
                                        )

In [ ]:
import os
import time
import pickle

CACHE_FILE = "./results/flash/data_sim_cache.pkl"
print(os.path.exists(CACHE_FILE))
# Try to load from pickle cache
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "rb") as f:
        cache = pickle.load(f)
    data_sim                    = cache["data"]
    data_gen_time               = cache["gen_time"]
    total_original_assSim_calls = cache["assSim_calls"]
    flash_1_range               = cache["flash_1_range"]
    flash_2_range               = cache["flash_2_range"]
    print(f"Loaded cache (shape={data_sim.shape}, sim calls={total_original_assSim_calls}, gen time={data_gen_time:.2f}s)")
else:
    # Define parameter ranges
    flash_1_range = np.linspace(1, 20, 7)
    flash_2_range = np.linspace(1, 20, 7)

    start_time = time.time()
    data_sim = []
    for flash_1 in flash_1_range:
        for flash_2 in flash_2_range:
            print(flash_1, flash_2)
            x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
            f, g = assSim.run_obj(x_unflat)
            data_sim.append([flash_1, flash_2, f, g])

    data_sim = np.array(data_sim, dtype=float)
    data_gen_time               = time.time() - start_time
    total_original_assSim_calls = len(data_sim)

    # Build cache dict
    cache = {
        "data": data_sim,
        "gen_time": data_gen_time,
        "assSim_calls": total_original_assSim_calls,
        "flash_1_range": flash_1_range,
        "flash_2_range": flash_2_range,
    }
    # Write to pickle
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(cache, f)
    print(f"Simulation done in {data_gen_time:.2f}s → saved to {CACHE_FILE}")

print("data shape:", data_sim.shape)
print("sim time (s):", data_gen_time)
print("assSim calls:", total_original_assSim_calls)


In [ ]:
print(len(data_sim))
scaler = TorchMinMaxScaler((-1, 1), 
                           max_vals=[20, 6],
                           min_vals=[1, 1],
                           min_y=[1000, -100],
                           max_y=[4e5 ,30],
                           scale_y=True)
#scale data_sim 
data_sim = data_sim.astype(float)
print("dtype:", data_sim.dtype)
print("first 5 rows:\n", data_sim[:5])

data_sim_x = torch.tensor(data_sim[:,:2],dtype=torch.float32)
data_sim_xscaled, data_sim_yscaled = scaler.transform(data_sim[:, :2], data_sim[:, 2:])

#recombine the data
data_sim_scaled = np.column_stack([data_sim_xscaled, data_sim_yscaled])

# **Initialize Model & Datasets**
dataset = DynamicDataset(data_sim_scaled)
model = MLP(2, [20,20, 20], 2)

In [ ]:
problem = Refrig2Drumproblem(model, scaler)
def my_ga_factory(it, prev_pop):
    if prev_pop is None:
        return GA(
            pop_size=1000,
            sampling=LHS(),
            eliminate_duplicates=False
        )
    else:
        return GA(
            pop_size=1000,
            sampling=ResumeFromPopulation(prev_pop),
            eliminate_duplicates=False
        )

In [ ]:
out = optimize_surr_ga(model=model,
                   dataset=dataset,
                   assSim=assSim,
                   problem=problem,
                   algo_factory=my_ga_factory,
                   lrs={'first':1e-4, 'others':1e-4},
                   epochs={'first':3000, 'others':500},
                   scaler=scaler,
                   device='cpu',
                   iter=10,
                   print_loss=True,
                   print_it_data=True,
                   pop_size=1000,
                   n_gen = 3,
                   new_data_size=15,
                   batch_size=128
                   )

In [ ]:
import os
import datetime
import pickle

print(os.getcwd())
# Get current notebook name manually (or hardcode it here)
notebook_name = "flash"  # or os.path.basename(__file__).replace(".py", "")

# Get timestamp
now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Construct path: results/<notebook_name>/run_<timestamp>/
base_dir = os.path.join("../NSGA_nn/results", notebook_name)
save_dir = os.path.join(base_dir, f"run_{now}")
os.makedirs(save_dir, exist_ok=True)
print(save_dir)

print(f"Results will be saved in: {save_dir}")

# Save out.pkl
out["data_gen_time"] = data_gen_time
out["total_original_assSim_calls"] = total_original_assSim_calls
with open(os.path.join(save_dir, "out.pkl"), "wb") as f:
    pickle.dump(out, f)

# Save scaler
with open(os.path.join(save_dir, "scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

# Save dataset
with open(os.path.join(save_dir, "dataset.pkl"), "wb") as f:
    pickle.dump(dataset, f)

print(f"Saved all results in {save_dir}")
